In [ ]:
import matplotlib.pyplot as plt
import ipdb
import os, glob, shutil
import os.path as osp
import numpy as np
import tqdm
from pydicom import dcmread
from PIL import Image

def sorted_list(path): 
    
    """ function for getting list of files or directories. """
    
    tmplist = glob.glob(path) # finding all files or directories and listing them.
    tmplist.sort() # sorting the found list
    
    return tmplist

def crop(ds,patch_size,stride):
    patches = []
    for left in range(0, ds.shape[0] - patch_size + 1, stride):
        for top in range(0, ds.shape[1] - patch_size + 1, stride):
            patches.append(ds[left: left + patch_size, top: top + patch_size])
    return patches

def arr_size(arr,size):
    s=[]
    for i in range(0,int(len(arr))+1,size):
        c=arr[i:i+size]
        s.append(c)
    return s

def make_dir(path, refresh=False):
    
    """ function for making directory (to save results). """
    
    try: os.mkdir(path)
    except: 
        if(refresh): 
            shutil.rmtree(path)
            os.mkdir(path)


data_root='/home/user/Munish/ASCON/CT_data'
# patient_root='/data/zhchen/aapm_challenge/data2d/'
# patient_list=sorted_list(patient_root+i+'/*')

#data_split={'train':[0:8],'test':[8:10]}
"""
train:L067,L096,L109,L143,L192,L286,L219,L310
test: L333,L506
"""

dataset_name = '/home/user/Munish/ASCON/mayo_data'
threshold = 0.85



patch_size=512
stride=512


for phase in ['train','test']:
    full_patches=[]
    quarter_patches=[]
    save_root_q=dataset_name+'/'+phase+'/'+'quarter_1mm'
    save_root_f=dataset_name+'/'+phase+'/'+'full_1mm'
    make_dir(save_root_q, refresh=True)
    make_dir(save_root_f, refresh=True)
    print('generate {} data...'.format(phase))
    for i in ['full_1mm','quarter_1mm']:
        print('generate {} data...'.format(i))
        if phase=='train':
            patient_list=sorted_list(data_root+'/'+i+'/*')[0:8]
        if phase=='test':
            patient_list=sorted_list(data_root+'/'+i+'/*')[8:10]
        for patient_id,patient_path in enumerate(tqdm.tqdm(patient_list)):
            ct_list = sorted_list(patient_path+'/'+i+'/*')
            for path in ct_list:
                #ipdb.set_trace()
                img=dcmread(path)
                patient_id=img.SliceLocation
                img_array=img.pixel_array.astype(np.float32)
                patches=crop(img_array,patch_size,stride)
                if i =='quarter_1mm':
                    quarter_patches.extend(patches)
                if i =='full_1mm':
                    full_patches.extend(patches)
                    
    print(len(quarter_patches))
    print(len(full_patches))

    count=0
    for k in tqdm.trange((len(full_patches))):
        #print(black_percent)
        black_percent = np.mean(np.clip(full_patches[k] - 1024, -500, 2000) == -500)
        if black_percent < threshold:
            count+=1
            #print(black_percent)
            #import ipdb
            #ipdb.set_trace()
            q_patch = np.array(quarter_patches[k]).astype(np.uint16).reshape(( 1,patch_size, patch_size))
            f_patch = np.array(full_patches[k]).astype(np.uint16).reshape(( 1,patch_size, patch_size))
            np.save(save_root_q+'/'+phase+'-quarter_1mm-'+'%05d' % k, np.array(q_patch).astype(np.uint16))
            np.save(save_root_f+'/'+phase+'-full_1mm-'+'%05d' % k, np.array(f_patch).astype(np.uint16))
    print(count)
    print('complete save {} patches...'.format(phase))
    

    